# Credit Risk Probability of Default - Data Exploration

In [1]:
# importing libraries and packages
import pandas as pd

In [2]:
dataset = pd.read_csv('./data/raw/Lending_Club_Loan_Data.csv')

C:\Users\Piyush\AppData\Local\Temp\ipykernel_10808\2745060242.py:1: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('./data/raw/Lending_Club_Loan_Data.csv')


In [ ]:
dataset.to_csv('./data/raw/loan_dataset.csv.gz',index = False, compression = 'gzip')